## Preprocessing and Feature Extraction

We want to put data into machine learning models.

But: Data might not be given in a format that can directly be given to our models. (E.g., how do you represent text? How do we transform the text into "numbers" that we can feed to our models?)

In general, we need to convert our data items (words, sentences, images, videos) into vectors or matrices (or sequences of them).


#### Example &ndash; Sentiment dataset

Today we will use the [Sentiment Polarity Dataset](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains 5331 positive and 5331 negative sentences from movie reviews, separated into two files (the positive reviews in one file, and the negative reviews in another one). Download the data from [here](http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz), unpack it, and put it in the same folder as this Jupyter Notebook.

(I'm probably not supposed to redistribute the data, but just in case anyone has problems to deal with .tar.gz files, I also left in the Downloads Folder a .zip file containing the exact same data)

Let's take a look at how the data looks like, for both the positive and negative reviews...

In [7]:
# First let's load some libraries that we will use in the code below
import nltk
import re
import random
import numpy as np

In [8]:
# Opens file of positive polarity data
with open('rt-polaritydata/rt-polaritydata/rt-polarity.pos', 
          'r', encoding='latin-1') as f:
    positives = [sentence.strip() for sentence in f]

# Prints the first 3 sentences, separated by "--"
for i in [0,1,2]:
    print(positives[i])
    print('--')

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
--
the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .
--
effective but too-tepid biopic
--


In [9]:
# Let's do the same with the negative polarity data
with open('rt-polaritydata/rt-polaritydata/rt-polarity.neg',
          'r', encoding='latin-1') as f:
    negatives = [sentence.strip() for sentence in f]

# Let's also print the first 3 sentences
for i in [0,1,2]:
    print(negatives[i])
    print('--')

simplistic , silly and tedious .
--
it's so laddish and juvenile , only teenage boys could possibly find it funny .
--
exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable .
--


### Preprocessing in NLP

We generally need to convert sentences to sequences of manageable "concepts" (e.g. words, stems, lemmas).

This can include:
- Tokenization
- Filtering
- Lemmatization
- POS tagging
- Lower case or other transformations
- Vocabulary building
- Restructuring the data

What exactly we do here depends on the method we want to use for processing the data afterwards.

#### Example &ndash; Sentiment dataset

Coming back to the sentiment dataset, let's select a random sentence from the dataset and illustrate how we would preprocess the sentence:

In [10]:
# Load a random sentence
# (notice that `positives + negatives` is the concatenation of the two lists)
example_sentence = random.choice(positives + negatives)

# Let's print the chosen sentence
print(example_sentence)

whether writer-director anne fontaine's film is a ghost story , an account of a nervous breakdown , a trip down memory lane , all three or none of the above , it is as seductive as it is haunting .


In [11]:
# Tokenization
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

print(tokenize(example_sentence))

['whether', 'writer-director', 'anne', 'fontaine', "'s", 'film', 'is', 'a', 'ghost', 'story', ',', 'an', 'account', 'of', 'a', 'nervous', 'breakdown', ',', 'a', 'trip', 'down', 'memory', 'lane', ',', 'all', 'three', 'or', 'none', 'of', 'the', 'above', ',', 'it', 'is', 'as', 'seductive', 'as', 'it', 'is', 'haunting', '.']


In [12]:
# Filtering
def filter_(tokenized_sentence):
    # The regular expression will filter out anything that
    # is not composed of letters (i.e., contains "," or ".",
    # or "&", or "?", etc.)
    return [word for word in tokenized_sentence
                  if re.match(r'^[\w]+$', word)]

print(filter_(tokenize(example_sentence)))

['whether', 'anne', 'fontaine', 'film', 'is', 'a', 'ghost', 'story', 'an', 'account', 'of', 'a', 'nervous', 'breakdown', 'a', 'trip', 'down', 'memory', 'lane', 'all', 'three', 'or', 'none', 'of', 'the', 'above', 'it', 'is', 'as', 'seductive', 'as', 'it', 'is', 'haunting']


In [13]:
# Of course, we can do the same with all words in our dataset. For example...
[word for sent in positives+negatives
      for word in filter_(tokenize(sent))]

['the',
 'rock',
 'is',
 'destined',
 'to',
 'be',
 'the',
 '21st',
 'century',
 'new',
 'conan',
 'and',
 'that',
 'he',
 'going',
 'to',
 'make',
 'a',
 'splash',
 'even',
 'greater',
 'than',
 'arnold',
 'schwarzenegger',
 'van',
 'damme',
 'or',
 'steven',
 'segal',
 'the',
 'gorgeously',
 'elaborate',
 'continuation',
 'of',
 'the',
 'lord',
 'of',
 'the',
 'rings',
 'trilogy',
 'is',
 'so',
 'huge',
 'that',
 'a',
 'column',
 'of',
 'words',
 'can',
 'not',
 'adequately',
 'describe',
 'peter',
 'jackson',
 'expanded',
 'vision',
 'of',
 'j',
 'r',
 'r',
 'tolkien',
 'effective',
 'but',
 'biopic',
 'if',
 'you',
 'sometimes',
 'like',
 'to',
 'go',
 'to',
 'the',
 'movies',
 'to',
 'have',
 'fun',
 'wasabi',
 'is',
 'a',
 'good',
 'place',
 'to',
 'start',
 'emerges',
 'as',
 'something',
 'rare',
 'an',
 'issue',
 'movie',
 'that',
 'so',
 'honest',
 'and',
 'keenly',
 'observed',
 'that',
 'it',
 'does',
 'feel',
 'like',
 'one',
 'the',
 'film',
 'provides',
 'some',
 'great'

In [14]:
# And if we want to know the most frequent words, we already know what to use
words = nltk.FreqDist([word for sent in positives for word in filter_(tokenize(sent))])

In [15]:
words.most_common(20)

[('the', 5060),
 ('a', 3845),
 ('and', 3554),
 ('of', 3312),
 ('to', 1970),
 ('is', 1776),
 ('it', 1674),
 ('that', 1357),
 ('in', 1340),
 ('film', 899),
 ('with', 884),
 ('as', 880),
 ('but', 784),
 ('an', 758),
 ('its', 700),
 ('for', 674),
 ('this', 667),
 ('you', 636),
 ('movie', 538),
 ('on', 424)]

In [16]:
# If we want only the words, without the frequency, we can just take
# the first element of the tuple
[i[0] for i in words.most_common(20)]

['the',
 'a',
 'and',
 'of',
 'to',
 'is',
 'it',
 'that',
 'in',
 'film',
 'with',
 'as',
 'but',
 'an',
 'its',
 'for',
 'this',
 'you',
 'movie',
 'on']

As part of our preprocessing, we will want to create a "vocabulary". This vocabulary will be typically composed of the most common words in our text. The size of this vocabulary is arbitrary, and depends on things like our application, the amount of data we have, or the amount of resources our application is supposed to use.

We already know how to take the most common words. Now we need only two other things:

 * A symbol for the "other words", not the most common;
 * A numerical ID for each word.

Let's put this all into a function

In [17]:
# Receives
#  * A set of sentences
#  * A vocabulary size
#  * The "Unknown Symbol"
# Returns
#  * A set of words
#  * A dictionary mapping each word to an ID
def build_vocabulary(sentences, vocabulary_size, unknown_symbol):
    
    # We already ran this part above
    words = nltk.FreqDist([word for sent in sentences for word in filter_(tokenize(sent))])
    
    # Creates a list with all the words.
    vocabulary = [item[0] for item in words.most_common(vocabulary_size)]
    
    # Inserts the `unknown_symbol` in the first position
    vocabulary = [unknown_symbol] + vocabulary
    
    # Creates a mapping "word --> index"
    word_to_idx = {word: index for index, word in enumerate(vocabulary)}

    # Returns a set of words and the dictionary
    return set(vocabulary), word_to_idx

Now think again what we have achieved. Given the dataset, we created a function that is now able to get the most common words of this dataset and put it in a data structure (our "vocabulary").

I understand that this function may be still a little hard to understand. Maybe it will become a little clearer if we take a look at how this vocabulary looks like. Let's apply this function to our dataset.

In [18]:
# Gets all sentences
all_sents = positives+negatives

# Calls `build_vocabulary` on all sentences
vocabulary_set, word_to_idx = build_vocabulary(sentences=all_sents,
                                               vocabulary_size=200,
                                               unknown_symbol="<UNK>")

Pay attention to the two things that the function returned. The first element, called `vocabulary_set`, is a `set` containing each one of the words in the vocabulary. In `build_vocabulary()`, we chose these to be the most common words in the dataset (we passed `vocabulary_size=200` to choose only the 200 most common words). Let's take a look at these words:

In [19]:
vocabulary_set

{'<UNK>',
 'a',
 'about',
 'action',
 'after',
 'all',
 'almost',
 'also',
 'american',
 'an',
 'and',
 'another',
 'any',
 'are',
 'as',
 'at',
 'audience',
 'bad',
 'be',
 'because',
 'been',
 'being',
 'best',
 'better',
 'between',
 'big',
 'both',
 'but',
 'by',
 'ca',
 'can',
 'cast',
 'character',
 'characters',
 'comedy',
 'comes',
 'could',
 'de',
 'despite',
 'director',
 'do',
 'documentary',
 'does',
 'down',
 'drama',
 'end',
 'enough',
 'entertaining',
 'even',
 'ever',
 'every',
 'family',
 'far',
 'feel',
 'feels',
 'few',
 'film',
 'films',
 'first',
 'for',
 'from',
 'full',
 'fun',
 'funny',
 'get',
 'go',
 'good',
 'great',
 'had',
 'hard',
 'has',
 'have',
 'he',
 'heart',
 'her',
 'here',
 'his',
 'hollywood',
 'how',
 'humor',
 'i',
 'if',
 'in',
 'interesting',
 'into',
 'is',
 'it',
 'its',
 'itself',
 'just',
 'kind',
 'less',
 'life',
 'like',
 'little',
 'long',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'makes',
 'man',
 'many',
 'may',
 'me',
 'might',
 

You can also see that `'<UNK>'` is a word in the vocabulary. All the "rest" of the words, i.e., those that were not among the 200 most common, were grouped together into the "UNK" character. We are basically saying we don't care about those other words. They are probably not very informative.

Now let's take a look at the other data structure output by the vocabulary: the `word_to_idx` dictionary. It has a mapping word-to-number, where each word in our dictionary gets assigned a different number:

In [20]:
word_to_idx

{'<UNK>': 0,
 'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'it': 7,
 'that': 8,
 'in': 9,
 'as': 10,
 'but': 11,
 'film': 12,
 'with': 13,
 'this': 14,
 'for': 15,
 'its': 16,
 'movie': 17,
 'an': 18,
 'you': 19,
 'be': 20,
 'on': 21,
 'not': 22,
 'by': 23,
 'one': 24,
 'about': 25,
 'are': 26,
 'has': 27,
 'more': 28,
 'like': 29,
 'at': 30,
 'from': 31,
 'than': 32,
 'all': 33,
 'have': 34,
 'his': 35,
 'i': 36,
 'so': 37,
 'if': 38,
 'or': 39,
 'story': 40,
 'what': 41,
 'there': 42,
 'too': 43,
 'who': 44,
 'just': 45,
 'does': 46,
 'into': 47,
 'most': 48,
 'out': 49,
 'no': 50,
 'much': 51,
 'even': 52,
 'good': 53,
 'up': 54,
 'will': 55,
 'can': 56,
 'comedy': 57,
 'time': 58,
 'some': 59,
 'characters': 60,
 'he': 61,
 'only': 62,
 'little': 63,
 'way': 64,
 'their': 65,
 'do': 66,
 'funny': 67,
 'make': 68,
 'they': 69,
 'enough': 70,
 'been': 71,
 'very': 72,
 'we': 73,
 'your': 74,
 'never': 75,
 'when': 76,
 'director': 77,
 'makes': 78,
 'would': 79,
 'may

Of course, the "UNK" word also got assigned a number. It is common to assign 0 to it.

In [21]:
word_to_idx['<UNK>']

0

Finally, with a vocabulary in hand, we can "preprocess" a sentence by transforming each of our words into a number:

In [22]:
filtered_sentence = filter_(tokenize(example_sentence))

preprocessed = []
for word in filtered_sentence:
    if word in vocabulary_set:
        preprocessed.append(word_to_idx[word])
    else:
        preprocessed.append(0)

In [23]:
preprocessed

[0,
 0,
 0,
 12,
 6,
 2,
 0,
 40,
 18,
 0,
 4,
 2,
 0,
 0,
 2,
 0,
 166,
 0,
 0,
 33,
 0,
 39,
 0,
 4,
 1,
 0,
 7,
 6,
 10,
 0,
 10,
 7,
 6,
 0]

Of course, we can use list comprehensions instead:

In [24]:
[word_to_idx[word] if word in vocabulary_set else 0
            for word in filtered_sentence]

[0,
 0,
 0,
 12,
 6,
 2,
 0,
 40,
 18,
 0,
 4,
 2,
 0,
 0,
 2,
 0,
 166,
 0,
 0,
 33,
 0,
 39,
 0,
 4,
 1,
 0,
 7,
 6,
 10,
 0,
 10,
 7,
 6,
 0]

And, to avoid having to write this again and again, we can put this code into a function. Given a sentence and a vocabulary, this function transforms the sentence into a sequence of numbers.

In [25]:
# Convert sentence to sequence of integers
def preprocess(sentence, word_to_idx, vocabulary_set):
    # `else 0` assumes that first vocabulary item is the "unknown word"
    return [word_to_idx[word] if word in vocabulary_set else 0
            for word in filter_(tokenize(sentence))]

In [26]:
example_sentence

"whether writer-director anne fontaine's film is a ghost story , an account of a nervous breakdown , a trip down memory lane , all three or none of the above , it is as seductive as it is haunting ."

In [27]:
preprocess(example_sentence, word_to_idx, vocabulary_set)

[0,
 0,
 0,
 12,
 6,
 2,
 0,
 40,
 18,
 0,
 4,
 2,
 0,
 0,
 2,
 0,
 166,
 0,
 0,
 33,
 0,
 39,
 0,
 4,
 1,
 0,
 7,
 6,
 10,
 0,
 10,
 7,
 6,
 0]

### Feature extraction

Ok... so now we transformed our words into numbers. Unfortunately, this is often still not enough.

For many models, you will still do some sort of _feature extraction_ after preprocessing.

Roughly speaking, a _feature_ is some measurable characteristic that you can observe in given items. There are many different examples of features in the context of NLP. For example, for sentences you could extract features like...:

- _number of characters_
- total number of _words with negative sentiment_ (based on a list of negative words)
- counts of a certain _POS-tag_ (e.g., counts of nouns, or of verbs)
- _Occurrence count_ of some fixed word

A set of such numeric features is then referred to as a _feature vector_. This is what will finally be used as input to machine learning models.

One common type of feature vector are the so-called _"bag-of-words (BoW)" features_. To generate them, we follow the steps below:

1. You first take a vocabulary (typically from the most common words from your dataset). Let's say your vocabulary consists of the words $w_1, \ldots, w_v$.
2. Now, for each sentence,
  1. You count how many times each of the vocabulary words occurs in the sentence. Let's say word $w_i$ occurs $n_i$ times in the sentence.
  2. The BoW feature vector of the sentence is then given by $(n_1, \ldots, n_v)$.


#### Example &ndash; Sentiment dataset

Let's compute bag-of-words representations for sentences of our given dataset.

We wrap it all up so we can re-use it in a more compact way:

In [28]:
# Transforms a sentence into a bag of words
def bag_of_words(sentence, vocabulary_set, word_to_idx):
    
    # Generates a vector of zeros
    bow_vector = np.zeros(len(vocabulary_set), dtype="uint8")

    # Transforms the sentence into a sequence of numbers
    indices = preprocess(sentence, word_to_idx, vocabulary_set)
    
    # Counts the occurrence of each indice
    for idx in indices:
        bow_vector[idx] += 1
    return bow_vector

In [29]:
# Just recapitulating the example sentence
example_sentence

"whether writer-director anne fontaine's film is a ghost story , an account of a nervous breakdown , a trip down memory lane , all three or none of the above , it is as seductive as it is haunting ."

In [30]:
print(bag_of_words(example_sentence, vocabulary_set, word_to_idx))

[15  1  3  0  2  0  3  2  0  0  2  0  1  0  0  0  0  0  1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  1  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0]


In [25]:
# There is an actually nice way of doing it seriously using "clojures",
# an advanced Python feature that we won't discuss here. I'm leaving it here
# in case anyone is interested...
#
# def feature_function(sentences, vocabulary_size=200):
#     """This returns a function that takes a sentence and returns a
#        feature representation of the sentence."""
#     vocabulary_set, word_to_ix = build_vocabulary(sentences, vocabulary_size)
#     
#     def bow_features(sentence):
#         bow_vector = np.zeros(len(vocabulary_set), dtype="uint8")
#         
#         for ix in preprocess(sentence, word_to_ix, vocabulary_set):
#             bow_vector[ix] += 1
#         return bow_vector
#     
#     return bow_features
#
#
# Create a function to extract features for sample of the given dataset
# bow_representation = feature_function(positives+negatives, vocabulary_size=200)
#
# This can then be used like this...
# bow_representation('my new sentence')

In [31]:
sentences = ["The quick brown fox jumped over the lazy dog. The dog was lazy and therefore didn't care about it"]

In [45]:
vocabulary_set, word_to_idx = build_vocabulary(sentences, 3, "<UNK>")

In [46]:
word_to_idx

{'<UNK>': 0, 'The': 1, 'lazy': 2, 'dog': 3}

In [47]:
vocabulary_set

{'<UNK>', 'The', 'dog', 'lazy'}

In [49]:
bag_of_words(sentences[0], vocabulary_set, word_to_idx)

array([13,  2,  2,  2], dtype=uint8)

In [27]:
bow = [13 , 1 , 2 , 1 , 0,  0]

In [28]:
nlp_labels = [1] * len(positives) + [-1] * len(negatives)

In [29]:
nlp_data = positives + negatives

In [30]:
processed_data = [bag_of_words(s, vocabulary_set, word_to_idx) for s in nlp_data]

### Nearest neighbor classifiers

One common strategy: For a new sample $d$, find the closest known sample $d_j$, the so-called "nearest neighbor" (NN) of $d$, and classify $d$ as $l_j$ (i.e. the same class as the nearest neighbor).

#### Formal description

$$f(d) = l_j,\quad j := \text{argmin}_i\, \text{distance}(d,d_i)$$
where $\text{distance}(d,d_i)$ describes the distance between $d$ and $d_i$ according to some distance metric.


#### Implementation

The library `sklearn` has a nearest neighbor classifier implemented in the `neighbors` package. Let's see how to use it...

In [31]:
import sklearn
import sklearn.neighbors

In [32]:
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5)

In [33]:
clf.fit(processed_data, nlp_labels)

KNeighborsClassifier()

In [34]:
sentence = "Wow this was awesome"
bow_sentence = bag_of_words(sentence, vocabulary_set, word_to_idx)

In [35]:
bow_sentence

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=uint8)

In [36]:
sentence2 = "Wow this was very bad horrible terrible"
bow_sentence2 = bag_of_words(sentence2, vocabulary_set, word_to_idx)

In [37]:
clf.predict([bow_sentence, bow_sentence2])

array([-1,  1])

In [38]:
test_sentences = [
    'This movie was super bad',
    'The movie was horrible and should not be ever watched again',
    'I did not like the movie',
    'Wow this was very bad horrible terrible',
    
    'This was an awesome movie',
    'This was a great movie',
    'I do not know if I will ever see a better movie in my life',
    'Wow this was awesome'
]

test_labels = [-1, -1, -1, -1, 1, 1, 1, 1]

In [39]:
processed_sents = [bag_of_words(s, vocabulary_set, word_to_idx) for s in test_sentences]

In [40]:
clf.predict(processed_sents)

array([-1, -1, -1,  1,  1,  1,  1, -1])

In [41]:
predicted = clf.predict(processed_sents)

In [42]:
expected = np.array([-1, -1, -1, -1, 1, 1, 1, 1])

In [43]:
predicted == expected

array([ True,  True,  True, False,  True,  True,  True, False])

In [44]:
comparison = predicted == expected

In [45]:
len(comparison[comparison == True])

6

In [46]:
len(comparison)

8

In [47]:
precision = len(comparison[comparison == True]) / len(comparison)

In [48]:
precision

0.75

In [58]:
sentences = ["The quick brown fox jumped over the lazy dog. The dog was lazy and therefore didn't care about it"]

In [59]:
vocabulary_set, word_to_idx = build_vocabulary(sentences, 3, '<UNK>')

In [60]:
vocabulary_set

{'<UNK>', 'The', 'dog', 'lazy'}

In [62]:
bag_of_words(sentences[0], vocabulary_set, word_to_idx)

array([13,  2,  2,  2], dtype=uint8)